<a href="https://colab.research.google.com/github/sabhi-29/Plasmid_dataset/blob/main/Plasmid_data_version_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Installing biopython first
!pip install biopython

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 26.1 MB/s eta 0:00:00


In [ ]:
# Importing neccesary libraries
import os
import numpy as np
import pandas as pd
from Bio import Entrez, SeqIO
from urllib.request import urlretrieve
import requests

In [ ]:
# Making function to create folders to store plasmid details for various hits
# separately
def create_new_folder(folder_name):
    # Check if the folder already exists
    if not os.path.exists(folder_name):
        # Create the folder

        try:
          os.mkdir(folder_name)
          return os.path.abspath(folder_name)
        except:
          if not os.path.exists("weird_name"):
            folder_name = "weird_name"
            os.mkdir(folder_name)
            return os.path.abspath(folder_name)
          else:
            print(f"Folder weird_name already exists.")
            return None
    else:
        print(f"Folder '{folder_name}' already exists.")
        return None

# Example usage
# folder_name = "new_folder"
# folder_path = create_new_folder(folder_name)
# if folder_path:
#     print(f"Folder '{folder_name}' created at {folder_path}")


In [ ]:
# Writing the function that will pull all the plasmid hits
def plasmid_data(plasmid_name_1, plasmid_email_id):

  # Making the folder for the plamsmid hits and storing it's path
  folder_name = plasmid_name_1
  if '.' in folder_name:
    folder_name = folder_name.split('.')
    temp_str = ''
    for sub_str in folder_name:
      temp_str += sub_str
    folder_name = temp_str
  path = create_new_folder(folder_name)
  print(f"Folder made for {folder_name}")

  # Setting up our query and accessing Entrez
  query = f"Plasmid {plasmid_name_1}"
  email = plasmid_email_id
  Entrez.email = email

  # Searching GenBank now
  handle = Entrez.esearch(db = 'nucleotide', term = query, retmax = 100)
  record_id = Entrez.read(handle)['IdList']
  handle.close()
  # print(record_id)
  # Now storing the data we got in form of a dataframe
  # First making the dictionary to store the data

  plasmid_data = {"Accession No.": [], 'Organism': [], 'Topology': [], 'Plasmid': [], 'Nucleotide Sequence': [],
                  'Locus Tag': [], 'Gene Location': [], 'Gene Product': [], 'Gene Sequence': []}

  # Going through each record we got from the entrez esearch
  for records in record_id:

    # Extracting the genbank record for the id
    genbank_record = Entrez.efetch(db = 'nucleotide', id = records, rettype = 'gb', retmode = 'text')
    # Reading the record
    gb_record = SeqIO.read(genbank_record, 'genbank')
    if (len(gb_record.id) != 0 and len(gb_record.description) != 0) and len(gb_record.seq) != 0:
      # We will first check if the sequence is of our plasmid of interest and complete or not, if yes then we proceed
      # Extracting Accession Number and description
      accession = [str(gb_record.annotations['accessions'][0])]
      description = gb_record.description

      ##### DEBUGGING PRINT STATEMENT ##############################
      print(f"Current hit: {plasmid_name_1} :- {description}")
      ################################################################


      # We are going to add the record only if it is complete and has the plasmid of interest
      if plasmid_name_1.lower() in description.lower() and 'complete sequence' in description.lower():
        organism = [str(gb_record.annotations["source"])]
        plasmid_name = [str(description.split("plasmid ")[-1].split(",")[0])]
        locus_tag = []
        topology = [str(gb_record.annotations['topology'])]
        gene_prdt = []
        gene_seq = []
        location = []

        ###########################################
        # Getting nucleotide sequence
        handle = Entrez.efetch(db="nucleotide", id=accession[0], rettype="fasta", retmode="text")
        fasta_sequence = handle.read()
        handle.close()
        fasta_seq_split = [x for x in fasta_sequence.split()[1:] if len(x) > 10] # To remove all the words before the sequence (get fasta file first then proceed)
        nuc_seq = ''
        for i in range(len(fasta_seq_split)):
          nuc_seq += fasta_seq_split[i]
        # print(nuc_seq)
        nucleotide_seq = [str(nuc_seq)]
        ###########################################################

        # Now we will access the record and store the bacteria name, accession number,
        # topology, plasmid, nucleotide sequence, locus tags for each gene present in it and thier sequence

        # Now that we have nucleotide of interest we can add it in the dataset
        for features in gb_record.features:
          if features.type == 'CDS':
            # In case translation, locus tag, gene product is missing then we simply leave that cell empty
            # print(features.qualifiers.keys())
            if 'locus_tag' in features.qualifiers.keys():
              locus_tag.append(str(features.qualifiers.get('locus_tag')[0]))
            else:
              locus_tag.append('NaN')
            if 'product' in features.qualifiers.keys():
              gene_prdt.append(str(features.qualifiers['product'][0]))
            else:
              gene_prdt.append('NaN')
            if 'translation' in features.qualifiers.keys():
              gene_seq.append(str(features.qualifiers['translation'][0]))
            else:
              gene_seq.append('NaN')
          if features.type == 'gene':
            location.append(str(features.location))

        # We will make the dataframe and store it only if all the columns are of the same length
        # print(len(locus_tag), len(locus_tag))
        if (len(locus_tag) == len(location) and len(locus_tag)!= 0):
          plasmid_data["Accession No."] = accession*len(locus_tag)
          plasmid_data["Organism"] = organism*len(locus_tag)
          plasmid_data["Topology"] = topology*len(locus_tag)
          plasmid_data["Plasmid"] = plasmid_name*len(locus_tag)
          plasmid_data["Nucleotide Sequence"] = nucleotide_seq*len(locus_tag)
          plasmid_data["Locus Tag"] = locus_tag
          plasmid_data["Gene Length"] = [len(x) for x in gene_seq]
          plasmid_data["Gene Sequence"] = gene_seq
          plasmid_data["Gene Product"] = gene_prdt
          plasmid_data['Gene Location'] = location
          # print(plasmid_data)
          # Making the dataframe
          dataframe = pd.DataFrame(plasmid_data)
          # Saving the dataframe
          dataframe.to_excel(f'{path}/{accession[0]}_{plasmid_name[0]}_data.xlsx', index = False)
          print(f"DataFrame made for {plasmid_name[0]} at {path}/{accession[0]}")
          dataframe.head()
        else:
          print("The dataset does not contain all the values")


In [ ]:
# # Downloading the list of plasmids in PLSDB Database

# url = 'https://github.com/sabhi-29/Plasmid_dataset/blob/4fd1d870cffd9c364a183e7dacf7508b9f962e8c/plsdb.xlsx'
# r = requests.get(url, allow_redirects=True)
# open(f'plsdb.xlsx', 'wb').write(r.content)




In [ ]:
# Reading the file into dataframe
plsdb_excel =  '/content/drive/MyDrive/plsdb_acc_num_data.xlsx'
df_plsdb = pd.read_excel(plsdb_excel, header=0, index_col=False, keep_default_na=True)

# Extracting the description list
descriptions = [x.split() for x in df_plsdb['Description_NUCCORE']]




In [ ]:
plasmids = []

for splits in descriptions:
  try:
    plasmids.append(splits[splits.index('plasmid')+1])
  except:
    continue
# print(len(plasmids))

In [ ]:
#### JUST TO SHOW THE OUTPUT OF THE ABOVE

plasmids = []

for splits in descriptions:
  try:
    plasmids.append(splits[splits.index('plasmid')+1])
  except:
    continue



Streaming output truncated to the last 5000 lines.
lpG27,
lpV47,
lpT28,
lpN31,
lpE27,
lpB58,
cp28,
megaplasmid,
CtrF-6068_plasmid,
CtrE-32931_plasmid,
pIEC338SC3,
pIEC338SC2,
pIEC338SCOX,
pSHB9,
plasmid4
plasmid3
plasmid2
plasmid1
QpH1,
unnamed2,
unnamed1,
Plasmid3
Plasmid2
Plasmid1
pFRP1-2,
pFRP1-1,
pLS1-1,
3,
2,
1,
pD36-4,
pD36-3,
pRAY*
pD36-1,
2,
1,
pTA78,
pTA69,
pTA16,
pTA14,
ABSP7_p5,
ABSP7_p4,
ABSP7_p3,
ABSP7_p2,
ABSP7_p1,
plasmid200,
plasmid100,
pFORC13,
pXAC33,
pXAC64,
lbp2,
lbp1,
pFORC4,
bpln_1p,
skipping ['Sphingopyxis', 'macrogoltabida', 'strain', '203', 'plasmid,', 'complete', 'sequence']
skipping ['Sphingopyxis', 'macrogoltabida', 'strain', '203', 'plasmid,', 'complete', 'sequence']
B,
A,
pNS6003,
pNS6002,
pNS6001,
pTrp,
pLeu,
skipping ['Bacillus', 'clausii', 'strain', 'ENTPro', 'plasmid,', 'complete', 'sequence']
skipping ['Altererythrobacter', 'atlanticus', 'strain', '26DY36', 'plasmid,', 'complete', 'sequence']
skipping ['Candidatus', 'Profftella', 'armatura', 'strain',

In [ ]:
# We can see that most of the entries in the list are what we want but not all
# so for now we will perform the FIRST ORDER FILTERING by removing all the single length entries

plasmid_list = [x[:-1] for x in plasmids if len(x) > 1]
print(plasmid_list[258:262])

['pRi_26-59', 'pTi_CFBP2788', 'pTi_Tun198', 'pTi_CFBP5767']


In [ ]:
# Testing the code

test_plasmids = plasmid_list[258:262]

for i in range(len(test_plasmids)):
  each_plasmid = str(test_plasmids[i])
  email_id = each_plasmid + '@gmail.com'
  print(f"Query running for {each_plasmid}. ({i+1}/{len(test_plasmids)})")
  plasmid_data(each_plasmid,email_id)


Query running for pRi_26-59. (1/4)
Folder made for pRi_26-59
pRi_26-59 Rhizobium rhizogenes strain 26-59 plasmid pRi_26-59, complete sequence
DataFrame made for pRi_26-59 at /content/pRi_26-59/NZ_KY000033
pRi_26-59 Rhizobium rhizogenes strain 26-59 plasmid pRi_26-59, complete sequence
The dataset does not contain all the values
Query running for pTi_CFBP2788. (2/4)
Folder made for pTi_CFBP2788
pTi_CFBP2788 Agrobacterium fabrum strain CFBP2788 plasmid pTi_CFBP2788, complete sequence
DataFrame made for pTi_CFBP2788 at /content/pTi_CFBP2788/NZ_KY000032
pTi_CFBP2788 Agrobacterium fabrum strain CFBP2788 plasmid pTi_CFBP2788, complete sequence
The dataset does not contain all the values
Query running for pTi_Tun198. (3/4)
Folder made for pTi_Tun198
pTi_Tun198 Agrobacterium fabacearum strain Tun198 plasmid pTi_Tun198, complete sequence
DataFrame made for pTi_Tun198 at /content/pTi_Tun198/NZ_KY000031
pTi_Tun198 Agrobacterium fabacearum strain Tun198 plasmid pTi_Tun198, complete sequence
The da

In [ ]:
# Running the code for the first 100 plasmids in the list

batches = [plasmid_list[i:i+100] for i in range(0,len(plasmid_list),100)]
print(f"Total batches: {len(batches)}")
i = 0
for each_batch in batches:
  print(f"Processing batch {i+1}/{len(batches)}.............")
  j = 0
  for each_plasmid in each_batch:
    each_plasmid = str(each_plasmid)
    email_id = each_plasmid + '@gmail.com'
    print(f"Query running for {each_plasmid}. ({j+1}/{len(each_batch)})")
    plasmid_data(each_plasmid,email_id)
    j += 1
  i += 1





Total batches: 130
Processing batch 1/130.............
Query running for p186-KPC. (1/100)
Folder made for p186-KPC
p186-KPC Aeromonas taiwanensis strain L186 plasmid p186-KPC, complete sequence
DataFrame made for p186-KPC at /content/p186-KPC/NZ_MH624130
p186-KPC Aeromonas taiwanensis strain L186 plasmid p186-KPC, complete sequence
DataFrame made for p186-KPC at /content/p186-KPC/MH624130
Query running for pEC07. (2/100)
Folder made for pEC07
pEC07 Acinetobacter sp. Res13-Abat-PEC07-P2-02 map unlocalized plasmid unnamednovel_0 unnamednovel_0_tig57, whole genome shotgun sequence
pEC07 Acinetobacter sp. Res13-Abat-PEC07-P2-02 seq_tig1, whole genome shotgun sequence
Query running for p838B-R. (3/100)
Folder made for p838B-R
p838B-R Escherichia coli strain 838B plasmid p838B-R, complete sequence
DataFrame made for p838B-R at /content/p838B-R/NZ_MH618673
p838B-R Escherichia coli strain 838B plasmid p838B-R, complete sequence
The dataset does not contain all the values
p838B-R Escherichia c

OSError: ignored

In [ ]:

# By previous PLSDB data we were getting many errors so we made a new dataset (which we see above)

# This list of Plasmids contain many items that do not give us any result upon searching. (Ex- p1249-D-T,p17829f).
# Some don't have "complete sequence" but complete cds (Ex- t-ST4).
# Some have "complete genome" and not complete sequence in their description. (Ex- p05-9280TC1).
# There are some other issues as well

# So for now, we will make a folder if we get empty desired result i.e- (have plasmid name and "complete sequence" meantioned in their description)
# and after traversing through all the plasmids we will delete the folders that are empty.

test_plasmids = plasmids[258:270]

for i in range(len(test_plasmids)):
  each_plasmid = str(test_plasmids[i])
  email_id = each_plasmid + '@gmail.com'
  print(f"Query running for {each_plasmid}. ({i}/{len(test_plasmids)})")
  plasmid_data(each_plasmid,email_id)





Query running for 38.27. (0/12)
Folder '3827' already exists.
Folder made for 3827
['1960491938', '1960491937', '1960491936', '1960491935', '1960491934', '1960491933', '1960491932', '1960491931', '1960491930', '1960491929', '1960491928', '1960491927', '1960491926', '1960491925', '1960491924', '1960491923', '1960491922', '1960491921', '1960491920', '1960491919', '1960491918', '1960491917', '1960491916', '1960491915', '1960491914', '1960491913', '1960491912', '1960491911', '1960491910', '1960491909', '1960491908', '1960491907', '1960491906', '1960491905', '1960491904', '1960491903', '1960491902', '1960491901', '1960491900', '1960491899', '1960491898', '1960491897', '1960491896', '1960491895', '1960491894', '1960491893', '1960491892', '1960491891', '1960491890', '1960491889', '1960491888', '1960491887', '1960491886', '1960491885', '1960491884', '1960491883', '1960491882', '1960491881', '1960491880', '1960491879', '1960491878', '1960491877', '1960491876', '1960491875', '1960491874', '19604

HTTPError: ignored